In [1]:
#!pip3 install tqdm
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from keras.callbacks import ModelCheckpoint


You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [2]:
#Code required only for Google Colab :

#!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
#from google.colab import auth
#auth.authenticate_user()
#from oauth2client.client import GoogleCredentials
#creds = GoogleCredentials.get_application_default()
#import getpass
#!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
#vcode = getpass.getpass()
#!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp6c3bf4zo/pubring.gpg' created
gpg: /tmp/tmp6c3bf4zo/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


UnboundLocalError: ignored

In [0]:
#Code only for google colab
#!mkdir -p DL
#!google-drive-ocamlfuse DL
#import sys
#sys.path.append('DL/Project')

In [4]:
#Code only for google colab
#!cd DL
#!ls

2discriminator1.h5  2generator1.h5  Aadhar card
2discriminator2.h5  2generator2.h5  discriminator60.h5
2discriminator3.h5  2generator3.h5  DL
2discriminator4.h5  2generator4.h5  FB_IMG_1490712712635.jpg
2discriminator5.h5  2generator5.h5  generator60.h5
2discriminator6.h5  2generator6.h5
2discriminator7.h5  2generator7.h5


In [0]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()


In [6]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')

# Scaling the range of the image to [-1, 1]
# and tanh restricts the weights in the range [-1, 1]
X_train = (X_train - 127.5) / 127.5

X_train.shape

(60000, 28, 28, 1)

In [0]:
generator = Sequential()
generator.add(Dense(1024, input_shape = (100,)))
generator.add(Activation('tanh'))
generator.add(Dense(128*7*7))
generator.add(BatchNormalization())
generator.add(Activation('tanh'))
generator.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, (5, 5), padding='same'))
generator.add(Activation('tanh'))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(1, (5, 5), padding='same'))
generator.add(Activation('tanh'))

In [0]:
discriminator = Sequential()
discriminator.add(Conv2D(64, (5, 5),input_shape=(28, 28, 1) , strides = (2,2)))
discriminator.add(BatchNormalization())
discriminator.add(Activation('tanh'))
discriminator.add(Conv2D(128, (5, 5) , strides = (2,2)))
discriminator.add(BatchNormalization())
discriminator.add(Activation('tanh'))
discriminator.add(Flatten())
discriminator.add(Dense(1024))
discriminator.add(Activation('relu'))
discriminator.add(Dense(1))
discriminator.add(Activation('sigmoid'))

In [0]:
import keras.backend as K
def wasserstein_loss(y_true, y_pred):
  return K.mean(y_true * y_pred)

In [0]:
generator.compile(loss = 'binary_crossentropy' , optimizer = 'adam')
discriminator.compile(loss = wasserstein_loss , optimizer = 'adam')
#checkpoint = ModelCheckpoint(filepath_dis, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [0]:
discriminator.trainable = False
ganInput = Input(shape=(100,))
# getting the output of the generator
# and then feeding it to the discriminator
# new model = D(G(input))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss=wasserstein_loss, optimizer=Adam())
#checkpoint = ModelCheckpoint(filepath_gen, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [0]:
#generator.load_weights('generator60.h5')
#discriminator.load_weights('discriminator60.h5')

In [0]:
def train(epoch=10, batch_size=128):
    batch_count = X_train.shape[0] // batch_size
    
    for i in range(epoch):
        print("<-------------------EPOCH " +str(i+1)+ " IS STARTING --------------------------->")
        for j in tqdm(range(batch_count)):
            # Input for the generator
            noise_input = np.random.normal(0 , 1 , size = (batch_size,100))
            
            # getting random images from X_train of size=batch_size 
            # these are the real images that will be fed to the discriminator
            image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]
            
            # these are the predicted images from the generator
            predictions = generator.predict(noise_input, batch_size=batch_size)
                        
            # the discriminator takes in the real images and the generated images
            X = np.concatenate([predictions, image_batch])
            
            # labels for the discriminator
            y_discriminator = [0]*batch_size + [1]*batch_size
            
            # Let's train the discriminator
            discriminator.trainable = True
            discriminator.fit(X, y_discriminator , verbose = 1)
            
            
            # Let's train the generator
            noise_input = np.random.rand(batch_size, 100)
            y_generator = [1]*batch_size
            discriminator.trainable = False
            gan.fit(noise_input, y_generator , verbose = 1)
        generator.save('generator_norm'+str(i+1)+'.h5')
        discriminator.save('discriminator_norm'+str(i+1)+'.h5')

In [0]:
train(300 , 128)

  0%|          | 0/468 [00:00<?, ?it/s]

<-------------------EPOCH 1 IS STARTING --------------------------->
Epoch 1/1
256/256 [==============================] - 0s 476us/step - loss: 3.0612e-07
Epoch 1/1
128/128 [==============================] - 0s 1ms/step - loss: 0.1748


  0%|          | 1/468 [00:00<02:14,  3.47it/s]

Epoch 1/1
256/256 [==============================] - 0s 431us/step - loss: 9.5217e-07
Epoch 1/1
128/128 [==============================] - 0s 1ms/step - loss: 0.2533


  0%|          | 2/468 [00:00<02:10,  3.57it/s]

...

Epoch 1/1
256/256 [==============================] - 0s 405us/step - loss: 9.8273e-14
Epoch 1/1
128/128 [==============================] - 0s 935us/step - loss: 1.6253e-10


 54%|█████▍    | 254/468 [01:03<00:53,  4.03it/s]

Epoch 1/1
256/256 [==============================] - 0s 402us/step - loss: 3.6772e-12
Epoch 1/1
128/128 [==============================] - 0s 941us/step - loss: 9.8617e-10


 54%|█████▍    | 255/468 [01:03<00:52,  4.03it/s]

Epoch 1/1
256/256 [==============================] - 0s 416us/step - loss: 4.6984e-12
Epoch 1/1
 32/128 [======>.......................] - ETA: 0s - loss: 2.9935e-10